In [8]:
import pandas as pd
import instance_generation
import  Vivian

In [9]:
filepath = "./../../Data/parc_features/parc_train_features.tsv"

In [10]:
df = pd.read_csv(filepath, delimiter='\t', index_col=0)

In [11]:
instance_output = instance_generation.collect_instances_main(df)

In [5]:
def seperate_instance_gold(instance_output):
    
    gold_label_list = list()
    pair_list = list()
    for instance_list in instance_output:
        gold_label_list.append(instance_list[2])
        pair_tupel = (instance_list[0], instance_list[1])
        pair_list.append(pair_tupel)

    #print(gold_label_list[:4])
    #print(pair_list[:4])
    
    return pair_list, gold_label_list

In [8]:
seperated = seperate_instance_gold(instance_output)

In [9]:
pair_list = seperated[0]
gold_label_list= seperated[1]

In [28]:
print(pair_list[:4])
for source, content in pair_list[:4]:
    #print(source)

[((58, 59), (57, 89)), ((60, 61), (57, 89)), ((65, 68), (57, 89)), ((72, 73), (57, 89))]
(58, 59)
(60, 61)
(65, 68)
(72, 73)


In [44]:
def create_pair_df(pair_list, df):
    
    source_list = list()
    content_list = list()
    
    for source, content in pair_list:
        source_range_list = list()
        content_range_list = list()
        
        for source_index in range(source[0], source[1]+1):
            source_range_list.append(source_index)
            
        for content_index in range(content[0], content[1]+1):
            content_range_list.append(content_index)
            
        
        collect_source_lemma = list()
        for item in source_range_list:
            collect_source_lemma.append(df.loc[item,'lemma'])
            source_list.append(collect_source_lemma)
            
        
        collect_content_lemma = list()
        for item in content_range_list:
            collect_content_lemma.append(df.loc[item, 'lemma'])
            content_list.append(collect_content_lemma)
        
        
    main_list = list(zip(source_list,content_list ))
        
    pair_df = pd.DataFrame(main_list, columns = ['item1_lemma', 'item2_lemma'])
    
    return pair_df

In [50]:
pair_df = create_pair_df(pair_list,df)

In [47]:
def create_instance_list(list_of_tuples,df):
    main_list = list()

    # Loop through instances meaning content and source span indices
    for instance in pair_list:
        # initiate instance_list and index_list
        attribution_indices = list()

        source, content = instance
        b_source, e_source = source
        b_content, e_content = content

        # For index of source
        for index in range(b_source, e_source+1):  
            attribution_indices.append(index)
        # for index of content
        for index in range(b_content, e_content+1):
            attribution_indices.append(index)
            
        # Find gap indices
        if b_source < b_content: 
            # If content follows source, gap is between last token of source and first of content
            gap_indices = [e_source, b_content]
        else:
            # If source follows content, then visa versa
            gap_indices = [e_content, b_source]

        list_tuple = (attribution_indices, gap_indices)
        main_list.append(list_tuple)
        
    return main_list

In [49]:
main_list = create_instance_list(pair_list, df)

In [ ]:
def find_sc_dist(pair_df, main_list):
    
    pair_df['s/c_distance'] = 'X'
    
    for list_tupel in main_list:
        instance_list = list_tupel[0]
        index_list = list_tupel[1]
        
        if instance_list[0] < instance_list[1]:
            distance = instance_list[1] - instance_list[0]
        else:
            distance = instance_list[0] - instance_list[1]

        pair_df.loc[index_list,'s/c_distance'] = distance

    return pair_df